In [1]:
import matplotlib.pyplot as plot
import os
import keyboard
import cv2
# import mediapipe as mp
import numpy as np
import numpy.ma as ma
import colorsys
import math
#import uuid
#import os    
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from queue import Queue

confidenceThreshold = 0.89

#custom width control
custom_width_g = 15
#Recommend running FTST on its own
simoultFTST = False;
#Sampling period(in num frames)
qMaxSize = 30


In [2]:
model = keras.models.load_model(os.getcwd()+'/model/modelA_batch32_epochstr15_n128.h5') #Loading model from dir
poses = np.array(['call_me',
                  'fingers_crossed',
                  'okay',
                  'paper',
                  'peace',
                  'rock',
                  'rock_on',
                  'scissor',
                  'thumbs',
                  'up'])

In [3]:
def rgb2hsv(rgb):
    colorInFloat = colorsys.rgb_to_hsv(rgb[0]/float(255), rgb[1]/float(255), rgb[2]/float(255))
    colorOut = (int(colorInFloat[0]*255), int(colorInFloat[1]*255), int(colorInFloat[2]*255))
    return colorOut

def boundTo255(numPassed, depth):
    return max(0, min(numPassed+depth, 255))

def customSkinTones(averageColorRGB, hueThresh, satThresh, valThresh):
    ## Returns Upper and Lower skin tones based on thresh and average skin color passed from face detection
    # Convert Color to HSV
    averageColorHSV = rgb2hsv(averageColorRGB)
    
    # Define range of acceptable skin tones
    
    #Default range
    # hueThresh = 10 
    # satThresh = 60
    # valThresh = 85
    
    ## Lower tone
    #Hue
    lowerHue = boundTo255(averageColorHSV[0],-1*hueThresh)
    #Sat
    lowerSat = boundTo255(averageColorHSV[1],-1*satThresh)
    #Val
    lowerVal = boundTo255(averageColorHSV[2],-1*valThresh)
    
    ## Upper Tone
    #Hue
    upHue = boundTo255(averageColorHSV[0],hueThresh)
    #Sat
    upSat = boundTo255(averageColorHSV[1],satThresh)
    #Val
    upVal = boundTo255(averageColorHSV[2],valThresh)
    
    # print('Lower:',lowerHue,lowerSat,lowerVal,'      Upper:',upHue,upSat,upVal)
    
    return np.array([upHue,upSat,upVal], dtype=np.uint8), np.array([lowerHue,lowerSat,lowerVal], dtype=np.uint8) #Upper then Lowers

def queue_ave(q):
    rebuilt_queue_list = []
    class_representations = np.zeros(20)
    qSize = q.qsize()
    while not q.empty():
        prediction = q.get()
        # Rebuild q
        # rebuilt_queue_list.append(prediction) # Enabled if there should not be a 1second sample time (immediate, no buffering)
        
        #get confidence from tuple
        confidence = prediction[1]
        #Adds the current highest confidence in class predictions to each class(row of the array)
        class_representations[prediction[0]] = class_representations[prediction[0]] + confidence
        
    #find highest prediction sum
    avgConfidence = np.amax(class_representations) / qSize
    # return the class with the highest confidence averaged over all predictions over some num frames, avg confidence 
    # print(np.argmax(class_representations), avgConfidence)
    return np.argmax(class_representations), avgConfidence



In [5]:
### Pose Estimation based on data Feed and skin tone values.
font=cv2.FONT_HERSHEY_SIMPLEX
currentPrediciton = 'Estimation:'
# Initializing a queue

q = Queue(maxsize = qMaxSize)

cap = cv2.VideoCapture(2)

while cap.isOpened():
    
    ret, frame = cap.read()
    
    if not ret:
            continue
            
    frame=cv2.flip(frame,1)#flip on vertical
    kernel = np.ones((3,3),np.uint8)

    #define region of interest
    roi = frame[350:545, 350:690] #240 x 195 : 65 x 80
    #add rectangle around region of interest
    cv2.rectangle(frame,(350,350),(590,545),(0,255,0),0)    
    
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV) 
    
    
    
    # Define default skin tones H S V
    lower_skin = np.array([0, 10, 60], dtype=np.uint8) # 0, 20, 70    89, 47, 42     0, 10, 60
    upper_skin = np.array([20, 150, 255], dtype=np.uint8) # 20, 255, 255     236, 188, 180      20, 150, 255
    
    # Create default mask
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    # Define custom skin tones H S V 
    #Skin Tone from Face
    averageColorRBG = (126, 80, 65)# RGB  (20, 44, 108)
    
    ##Simoultaneous FTST, not recommended. Calibrate using the method FTST instead.
    if simoultFTST:
        avgColor = (126, 80, 65)
        ##Face recognition to identify skin color
        #import OpenCV face detection model
        face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

        #custom width control
        custom_width = custom_width_g

        # Convert into grayscale
        grayScaleFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(grayScaleFrame, 1.1, 4)

            # Draw rectangle around the faces
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.rectangle(frame, (x+custom_width, y), (x+w-custom_width, y+h), (0, 255, 0), 1)
            try:
                sobel = sobelFilter(grayScaleFrame)
                cv2.imshow('Sobel Approx', sobel)
            except:
                pass
            #Slice command given
        if keyboard.is_pressed('s'):  # if key 'q' is pressed 
            # Detect faces
            faces = face_cascade.detectMultiScale(grayScaleFrame, 1.1, 4)
            # Draw rectangle around the faces
            for (x, y, w, h) in faces:
                #sliced frame
                try:
                    slicedSegment = frame[y:y+h,x+custom_width:x+w-custom_width]
                    img_temp = slicedSegment.copy()
                     #find avg pixel value for each face
                    img_temp[:,:,0], img_temp[:,:,1], img_temp[:,:,2] = np.average(slicedSegment, axis=(0,1))
                    avgColor = (int(img_temp[0][0][0]), int(img_temp[0][0][1]), int(img_temp[0][0][2]))
                    cv2.rectangle(frame, (x+custom_width, y), (x+w-custom_width, y+h), avgColor, -1)
                except:
                    print('err')
            averageColorRBG = avgColor
    ##End of simulFTST
        
    
    
    # customColorRGB = (116, 88, 83)
    #Default range
    hueThresh = 5 #Light Temperature variance, Multiple lightsources with different warmths - higher
    satThresh = 70 #Lighter skin requires higher saturation threshholds
    valThresh = 95 #Lighting variance of enviornment
    custom_upper_skin, custom_lower_skin = customSkinTones(averageColorRBG, hueThresh, satThresh, valThresh)
    
    custom_mask = cv2.inRange(hsv, custom_lower_skin, custom_upper_skin)
   
    ##Improve the mask
    #extrapolate the hand to fill dark spots within
    mask = cv2.dilate(mask,kernel,iterations = 4)
    #blur to reduce noise
    mask = cv2.GaussianBlur(mask,(5,5),100) 
    # custom_mask = cv2.dilate(custom_mask,kernel,iterations = 2)
    custom_mask = cv2.GaussianBlur(custom_mask,(5,5),100) 
    
    freshOutput = np.zeros((195,240,3), np.uint8)
    
    try:
        #find contours
        # contours,hierarchy= cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        custom_contours,custom_hierarchy= cv2.findContours(custom_mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        # find max area contour
        if custom_contours:
            cnt = max(custom_contours, key = lambda x: cv2.contourArea(x)) #if contours were found identify max inner area cnt 
            # cv2.drawContours(frame, cnt, -1, color=(255, 255, 255), thickness=cv2.FILLED)
            
            # approx the contour
            epsilon = 0.0005*cv2.arcLength(cnt,True)
            approx= cv2.approxPolyDP(cnt,epsilon,True)
            # cv2.drawContours(custom_mask, cnt, -1, (0, 255, 0), 3)
            # Optimal Output
            cv2.fillPoly(freshOutput, pts = [approx], color = (240, 240, 240))
            # cv2.fillPoly(custom_mask, pts = [approx], color = (240, 240, 240))
            freshOutput = cv2.GaussianBlur(freshOutput,(5,5),100) 
            freshOutput = cv2.erode(freshOutput,kernel,iterations = 4)
            
    except:
        pass
    
    ## Resize mask
    resizedMask = cv2.resize(freshOutput,(80,65), interpolation = cv2.INTER_AREA)
    # convert from Mat to numpy array
    np_image_data = np.asarray(resizedMask)
    np_final = np.expand_dims(np_image_data,axis = 0)
    #adds the None(on axis 0) dimensions that the model is expecting
    images = np.vstack([np_final])
    #adds the three channels of "color" (on axis 3) that the model is expecting
    
    try:
        # print(images.shape)
        #Predict
        prediction = model.predict(images)
        highestConfidence = np.amax(prediction)
        classOfHighest = poses[np.argmax(prediction)]
        # Adding to Confidence Queue
        q.put((np.argmax(prediction),np.amax(prediction))) #Adds the class index and confidence of the prediciton as a tuple to the q
        if not q.qsize() < qMaxSize:
            q.get()
            #Print
            result = queue_ave(q)
            if result[1] > confidenceThreshold:
                print(poses[result[0]],result[1])
                currentPrediciton = 'Estimation: ' + str(poses[result[0]]) + ' : ' + str(round(result[1], 4))
            else:
                currentPrediciton = 'Estimation: sampling...'
                
    except:
        pass
    
    cv2.putText(frame,currentPrediciton,(300,300), font, 1,(255,255,255),2, cv2.LINE_4)
    
    #show video feed
    cv2.imshow('Raw Video Input',frame)
    
    #Show defaul mask
    cv2.imshow('Default Skin Mask', mask)
    #Show mask
    cv2.imshow('Custom Skin Mask',custom_mask)
    #Show clean output
    cv2.imshow('Optimized',freshOutput)
    
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()
cap.release()    

rock 1.0
paper 0.9310344704266252


In [5]:
def faceToSkinTone():
    avgColor = (126, 80, 65)
    ##Face recognition to identify skin color
    #import OpenCV face detection model
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    #custom width control
    custom_width = custom_width_g

    #Webcam Capture Skeleton
    cap = cv2.VideoCapture(2) 
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue

        # Convert into grayscale
        grayScaleFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(grayScaleFrame, 1.1, 4)
        
        # Draw rectangle around the faces
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.rectangle(frame, (x+custom_width, y), (x+w-custom_width, y+h), (0, 255, 0), 1)
            try:
                sobel = sobelFilter(grayScaleFrame)
                cv2.imshow('Sobel Approx', sobel)
            except:
                pass
        #Slice command given
        if keyboard.is_pressed('s'):  # if key 'q' is pressed 
            # Detect faces
            faces = face_cascade.detectMultiScale(grayScaleFrame, 1.1, 4)
            # Draw rectangle around the faces
            for (x, y, w, h) in faces:
                # cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                #sliced frame
                slicedSegment = frame[y:y+h,x+custom_width:x+w-custom_width]
                img_temp = slicedSegment.copy()
                 #find avg pixel value for each face
                img_temp[:,:,0], img_temp[:,:,1], img_temp[:,:,2] = np.average(slicedSegment, axis=(0,1))
                avgColor = (int(img_temp[0][0][0]), int(img_temp[0][0][1]), int(img_temp[0][0][2]))
                print(avgColor)
                try:
                    cv2.rectangle(frame, (x+custom_width, y), (x+w-custom_width, y+h), avgColor, -1)
                except:
                    print('oopsies')

        cv2.imshow('Skin Color Calibration', frame)
        
        # cv2.imshow('BW Channel', grayScaleFrame)     
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    return avgColor

In [6]:
#alternate Sobel Filtering to better define contours, face ROI
def sobelFilter(frame):
    # Calculation of Sobelx
    sobelx = cv2.Sobel(frame,cv2.CV_64F,1,0,ksize=5)

    # Calculation of Sobely
    sobely = cv2.Sobel(frame,cv2.CV_64F,0,1,ksize=5)

    weightedFilterCombination = cv2.addWeighted(sobelx, 0.4, sobely, 0.3, 0)
    return weightedFilterCombination
# # Calculation of Laplacian
# laplacian = cv2.Laplacian(frame,cv2.CV_64F)
    

In [2]:
# Helpful for startup
# Grab indices of all Video Capture Devices (Webcams, virtual cams, ect.)
index = 0
arr = []
while True:
    cap = cv2.VideoCapture(index)
    if not cap.read()[0]:
        break
    else:
        arr.append(index)
    cap.release()
    index += 1
print(arr)

[0, 1, 2]


In [38]:
#Run stand alone instance to calibrate skin color
print(faceToSkinTone())

(47, 68, 77)
(70, 70, 87)
(83, 86, 109)
(83, 90, 117)
(84, 91, 118)
(83, 89, 116)
(83, 89, 116)
